# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

ModuleNotFoundError: No module named 'pymongo'

In [ ]:
!pip install pymongo

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# Access the specified database or create it if it doesn't exist
db = mongo['uk_food']

# Confirm that the database exists
print(f"Connected to the '{db}' database.")


In [ ]:
# review the collections in our new database
collections = db.list_collection_names()

# Print the collections
print("Collections in 'uk_food' database:", collections)

In [ ]:
print("List of databases:")
pprint(mongo.list_database_names())

In [ ]:
# review a document in the establishments collection
establishments_collection = db['establishments']
document = establishments_collection.find_one()
print("\nOne document from the establishments collection:")
pprint(document)

In [ ]:
# assign the collection to a variable
establishments = db['establishments']


## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [ ]:
# Create a dictionary for the new restaurant data
# 1. Add a new halal restaurant in Greenwich
new_restaurant = {
    "BusinessName": "New Halal Restaurant",
    "LocalAuthorityDistrict": "Greenwich",
    # Add other fields as needed
}

In [ ]:
# Insert the new restaurant into the collection
establishments_collection.insert_one(new_restaurant)

In [ ]:
# Check that the new restaurant was inserted
# Check that the new restaurant was inserted
result = establishments_collection.find_one({"BusinessName": "New Halal Restaurant"})

if result:
    print("New restaurant found:")
    print(result)
else:
    print("New restaurant not found.")


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [ ]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
# 2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen"
restaurant_type = establishments_collection.find_one({"BusinessType": "Restaurant/Cafe/Canteen"},
                                                    {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0})


3. Update the new restaurant with the `BusinessTypeID` you found.

In [ ]:
# Update the new restaurant with the correct BusinessTypeID
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen"
restaurant_type = establishments_collection.find_one({"BusinessType": "Restaurant/Cafe/Canteen"},
                                                    {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0})

if restaurant_type:
    # Update the new restaurant with the correct BusinessTypeID
    establishments_collection.update_one({"BusinessName": "New Halal Restaurant"},
                                         {"$set": {"BusinessTypeID": restaurant_type["BusinessTypeID"]}})
    print("Update successful.")
else:
    print("BusinessTypeID not found for 'Restaurant/Cafe/Canteen'. Please check your data.")


In [ ]:
# Confirm that the new restaurant was updated

updated_restaurant = establishments_collection.find_one({"BusinessName": "New Halal Restaurant"})

if updated_restaurant and "BusinessTypeID" in updated_restaurant:
    print("New restaurant updated successfully:")
    print(updated_restaurant)
else:
    print("New restaurant update failed.")


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [ ]:
# Find how many documents have LocalAuthorityName as "Dover"
# 4. Remove establishments in Dover Local Authority
num_documents_before = establishments_collection.count_documents({"LocalAuthorityDistrict": "Dover"})



In [ ]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments_collection.delete_many({"LocalAuthorityDistrict": "Dover"})

In [ ]:
# Check if any remaining documents include Dover
num_documents_after = establishments_collection.count_documents({})


In [ ]:
# Check that other documents remain with 'find_one'
# Check another document to ensure it remains unchanged
sample_document = establishments_collection.find_one({"SomeField": "SomeValue"})

if sample_document:
    print("Sample document found:")
    print(sample_document)
else:
    print("Sample document not found.")


5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [ ]:
# Change the data type from String to Decimal for longitude and latitude
# 5. Convert latitude and longitude to decimal numbers
establishments_collection.update_many({}, {
    "$set": {
        "Latitude": {"$toDouble": "$Latitude"},
        "Longitude": {"$toDouble": "$Longitude"}
    }
})

Use `update_many` to convert `RatingValue` to integer numbers.

In [ ]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [ ]:
# Change the data type from String to Integer for RatingValue
establishments_collection.update_many({}, {
    "$set": {
        "RatingValue": {"$toInt": "$RatingValue"}
    }
})

In [ ]:
# Check that the coordinates and rating value are now numbers
# Check that the coordinates and rating value are now numbers
sample_document_after_conversion = establishments_collection.find_one({"BusinessName": "New Halal Restaurant"})

if sample_document_after_conversion:
    latitude = sample_document_after_conversion.get("Latitude")
    longitude = sample_document_after_conversion.get("Longitude")
    rating_value = sample_document_after_conversion.get("RatingValue")

    print("Sample document after conversion:")
    print(f"Latitude: {latitude}, Type: {type(latitude)}")
    print(f"Longitude: {longitude}, Type: {type(longitude)}")
    print(f"RatingValue: {rating_value}, Type: {type(rating_value)}")
else:
    print("Sample document not found.")
